In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans

In [2]:
df=pd.read_csv('D:/news aggregator/uci-news-aggregator.csv')

In [3]:
df.head()

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


In [4]:
df2=df.iloc[:,[1,4]]

In [5]:
df2.head()

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",b
1,Fed's Charles Plosser sees high bar for change...,b
2,US open: Stocks fall after Fed official hints ...,b
3,"Fed risks falling 'behind the curve', Charles ...",b
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b


In [6]:
df2['CATEGORY']=df2['CATEGORY'].map({'b':0,'e':1,'m':2,'t':3})

C:\Users\Ishu\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
df2=df2.sample(frac=1).reset_index(drop=True)

In [8]:
#Split data in train and test sets

X=df2.iloc[:322937,0]
y=df2.iloc[:322937,1]
X_test=df2.iloc[322938:,0]
y_test=df2.iloc[322938:,1]

In [9]:
#import classifier and combine Tfidf transformer and Naive Bayes Classifier in a pipeline

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB())])

In [10]:
#fit classifier on train set

clf=text_clf.fit(X,y)
predicted = clf.predict(X_test)

In [11]:
#find the accuracy on test set

np.mean(predicted==y_test)

0.92321146751641014

In [12]:
#Using Linear SVM now

from sklearn.svm import LinearSVC
text_clf = Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',LinearSVC(loss='hinge',penalty='l2'))])

In [13]:
clf=text_clf.fit(X,y)

In [14]:
predicted = clf.predict(X_test)

In [15]:
np.mean(predicted==y_test)

0.95009097214543481

In [27]:
samp=["New Tesla Model hits the road"]
predict=clf.predict(samp)
print predict

[3]


In [19]:
from sklearn.neural_network import MLPClassifier
text_clf = Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',MLPClassifier(activation='relu',solver='adam'))])
clf=text_clf.fit(X,y)

In [20]:
predicted = clf.predict(X_test)

In [21]:
np.mean(predicted==y_test)

0.93845055839808611

In [10]:
#KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
transformer=Pipeline([('vect',HashingVectorizer()),('tfidf',TfidfTransformer())])
X_trans=transformer.fit_transform(X)
secTransformer=Pipeline([('svd',TruncatedSVD(10000))])
X_trans=secTransformer.fit_transform(X_trans)
K_clf= MiniBatchKMeans(n_clusters=3, init='k-means++')
clf=K_clf.fit(X_trans)
predicted=clf.predict(X_test)
predicted

MemoryError: 